In [7]:
import os
import sys
import json
import geopandas as gpd
sys.path.insert(0, r'../../../osm-api/') # Add path to quetzal
from main import *

In [8]:
road_folder = '../../inputs/road/'
zones_folder = '../../inputs/zones/'

# OVERPASS api fetch

In [9]:
bbox = [45.39, -74, 45.71, -73.33]
bbox = (*bbox,)

In [10]:
columns = ['highway', 'maxspeed', 'lanes', 'name', 'oneway', 'surface']
cycleway_columns = ['cycleway:both', 'cycleway:left','cycleway:right']
columns += cycleway_columns
columns += ['cycleway']

In [11]:
highway_list = ["motorway", "motorway_link", "trunk", "trunk_link", "primary", "primary_link", 
                  "secondary", "secondary_link", "tertiary", "tertiary_link", "residential","cycleway"]
                
cycleway_list = ["lane", "opposite", "opposite_lane", "track", "opposite_track", 
                "share_busway", "opposite_share_busway", "shared_lane"]

In [14]:
if True:
    links,nodes = osm_importer(bbox,highway_list,cycleway_list,road_folder)

OVERPASS Request ...
Convert to GeoPandas ...


/home/boivin/.pyenv/versions/3.8.6/envs/quetzal_env/lib/python3.8/site-packages/pandas/core/dtypes/cast.py:127: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)


Write (way.geojson) ...
Convert ways to links and node ...


# Process OSM network

In [15]:
links, nodes = get_links_and_nodes(os.path.join(road_folder, 'way.geojson'), split_direction=False)
nodes = nodes.set_crs(links.crs)

In [16]:
zones = gpd.read_file(zones_folder+'montreal.geojson').to_crs(4326)
links = gpd.sjoin(links, zones, how='inner', op='intersects').drop(columns = ['index_right','id'])


/home/boivin/.pyenv/versions/3.8.6/envs/quetzal_env/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3445: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


# process

In [17]:
#links,nodes = osm_simplify(links,nodes,highway_list,True)

In [18]:
CYCLEWAY_COLUMNS = ['cycleway:both', 'cycleway:left','cycleway:right']

In [21]:
links['cycleway'].unique()

array([nan, 'lane', 'shared_lane', 'no', 'track', 'crossing',
       'cyclestreet', 'share_busway', 'separate', 'opposite', 'shared',
       'use_sidepath'], dtype=object)

In [18]:
links = test_bicycle_process(links, CYCLEWAY_COLUMNS, highway_list)

# convert oneway to bool.
links = clean_oneway(links)

#remove string in maxspeed
links = clean_maxspeed(links)

#remove string in maxspeed
links = clean_lanes(links)

# make sure the geometry are in the right direction (a->b)
links = rectify_geometry_direction(links,nodes)

# remove duplicated links (a-b)
print("simplifying links ...")
links = drop_duplicated_links(links)

# simplify. remove deg 2 nodes when possible. group by oneway and highway to merge each links.
links = simplify(links)



['no', 'yes', 'shared']
['no', 'yes', 'shared']
['no', 'yes', 'shared']
['no', 'yes', 'shared']
0 geometry to inverse
simplifying links ...
375 links dropped
25420 deg 2 nodes
find path with large cutoff for 9  origins
find path with large cutoff for 4  origins
find path with large cutoff for 2  origins
find path with large cutoff for 2  origins
find path with large cutoff for 6  origins
find path with large cutoff for 2  origins
find path with large cutoff for 1  origins
find path with large cutoff for 3  origins
find path with large cutoff for 2  origins
find path with large cutoff for 18  origins
0 links were not merge because the oneway field is not the same
0 links were not merge because the highway field is not the same
0 merged_links unmerged because the geometry became a multilinestring


In [19]:
# split onwway into 2 links a-b, b-a
#links = split_oneway(links)

# Clean Cul de Sac
print("Remove Cul de Sac ...")
links, nodes = main_strongly_connected_component(links, nodes, split_direction=True)

Remove Cul de Sac ...


In [21]:
print('removing list in columns ...')
links['maxspeed'] = links['maxspeed'].apply(lambda x: process_list_in_col(x, float, np.nanmean))
links['lanes'] = links['lanes'].apply(lambda x: process_list_in_col(x, float, lambda x: np.floor(np.nanmean(x))))
if 'cycleway' in links.columns:
    # sort and take last. sorted = [no,shared,yes]. so yes or shared if there is a list
    links['cycleway'] = links['cycleway'].apply(lambda x: process_list_in_col(x,str,lambda x: np.sort(x)[-1]))
    links['cycleway_reverse'] = links['cycleway_reverse'].apply(lambda x: process_list_in_col(x,str,lambda x: np.sort(x)[-1]))

for col in ['highway','name','surface']:
    links[col] = links[col].apply(lambda x: remove_list_in_col(x,'first'))


# Fill NaN with mean values by highway
links = fill_na_col(links, 'highway', 'maxspeed', np.mean)
links = fill_na_col(links, 'highway', 'lanes', lambda x: np.floor(np.mean(x)))

# Add length
print("Write Links and Nodes ...")
epsg = get_epsg(nodes.iloc[0]['geometry'].y, nodes.iloc[0]['geometry'].x)
links['length'] = links.to_crs(epsg).length

# Add Time
links['time'] = links['length']/(links['maxspeed']*1000/3600)
links = links.rename(columns = {'maxspeed' : 'speed'})

# reindex and remove ununsed nodes
links = links.reset_index(drop=True)
links.index = 'road_link_'+links.index.astype(str)
nodes_set = set(links['a']).union(set(links['b']))
nodes = nodes.loc[list(nodes_set)].sort_index()

Write Links and Nodes ...


In [22]:
print('Adding elevation')
el_dict = get_elevation_from_srtm(nodes)
nodes['elevation'] = nodes.index.map(el_dict.get)
# incline from node a to b in deg. neg if going down (if b is lower dans a)
links['incline'] = calc_incline(links['a'].apply(lambda x: el_dict.get(x)).values,
                            links['b'].apply(lambda x: el_dict.get(x)).values,
                            links['length'].values)


Adding elevation
file save to /tmp


In [23]:
from road import get_columns_with_list

In [24]:
get_columns_with_list(links)

[]

In [ ]:
links.geometry = links.simplify(0.00005)

In [43]:
links.to_file(road_folder+'road_links.geojson',driver='GeoJSON')
nodes.to_file(road_folder+'road_nodes.geojson',driver='GeoJSON')